# Task 1: Đọc và tiền xử lý dữ liệu

In [2]:
def load_conllu(file_path):
    """
    Trả về: list_of_sentences, mỗi sentence là list of (word, upos)
    """
    sentences = []
    with open(file_path, 'r', encoding='utf-8') as f:
        sent = []
        for line in f:
            line = line.strip()
            if not line:
                if sent:
                    sentences.append(sent)
                    sent = []
                continue
            if line.startswith('#'):
                continue
            parts = line.split('\t')
            # guard: đôi khi có dòng không chuẩn
            if len(parts) < 5:
                continue
            word = parts[1]
            upos = parts[3]
            sent.append((word, upos))
        if sent:
            sentences.append(sent)
    return sentences


In [6]:
from collections import Counter

def build_vocab(sentences, min_freq=1, lowercase=True):
    word_counter = Counter()
    tag_set = set()
    for sent in sentences:
        for w, t in sent:
            if lowercase:
                w = w.lower()
            word_counter[w] += 1
            tag_set.add(t)
    # special tokens
    word_to_ix = {'<PAD>':0, '<UNK>':1}
    for w, c in word_counter.items():
        if c >= min_freq:
            word_to_ix[w] = len(word_to_ix)
    tag_to_ix = {tag: idx for idx, tag in enumerate(sorted(tag_set))}
    
    tag_to_ix['<PAD>'] = len(tag_to_ix)
    return word_to_ix, tag_to_ix

train_sentences = load_conllu(r'E:\Nam4\NLP\level 1\NLP_22001295_HUS\data\UD_English-EWT\en_ewt-ud-train.conllu')
dev_sentences = load_conllu(r'E:\Nam4\NLP\level 1\NLP_22001295_HUS\data\UD_English-EWT\en_ewt-ud-dev.conllu')

word_to_ix, tag_to_ix = build_vocab(train_sentences, min_freq=1, lowercase=True)

print("Vocab size:", len(word_to_ix))
print("Num tags:", len(tag_to_ix))


Vocab size: 17115
Num tags: 19


# Task 2 — Dataset & DataLoader (collate_fn)

In [7]:
import torch
from torch.utils.data import Dataset

class POSDataset(Dataset):
    def __init__(self, sentences, word_to_ix, tag_to_ix, lowercase=True):
        self.sentences = sentences
        self.w2i = word_to_ix
        self.t2i = tag_to_ix
        self.lowercase = lowercase
    
    def __len__(self):
        return len(self.sentences)
    
    def __getitem__(self, idx):
        sent = self.sentences[idx]
        words = []
        tags = []
        for w, t in sent:
            if self.lowercase:
                w = w.lower()
            w_idx = self.w2i.get(w, self.w2i.get('<UNK>'))
            t_idx = self.t2i[t]
            words.append(w_idx)
            tags.append(t_idx)
        return torch.tensor(words, dtype=torch.long), torch.tensor(tags, dtype=torch.long)


In [8]:
from torch.nn.utils.rnn import pad_sequence

def collate_fn(batch):
    words = [item[0] for item in batch]
    tags = [item[1] for item in batch]
    lengths = torch.tensor([len(x) for x in words], dtype=torch.long)
    words_padded = pad_sequence(words, batch_first=True, padding_value=0)  # <PAD> index = 0
    tags_padded = pad_sequence(tags, batch_first=True, padding_value=-100)  
    
    return words_padded, tags_padded, lengths


In [13]:
from torch.utils.data import DataLoader

train_dataset = POSDataset(train_sentences, word_to_ix, tag_to_ix)
dev_dataset = POSDataset(dev_sentences, word_to_ix, tag_to_ix)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, collate_fn=collate_fn)
dev_loader = DataLoader(dev_dataset, batch_size=64, shuffle=False, collate_fn=collate_fn)


# Task 3 — Xây dựng mô hình RNN

In [18]:
import torch.nn as nn

class SimpleRNNForTokenClassification(nn.Module):
    
    def __init__(self, vocab_size, embedding_dim, hidden_dim, num_tags, padding_idx=0):
        super(SimpleRNNForTokenClassification, self).__init__()
        
        # 1. Lớp Embedding: Chuyển đổi ID của từ thành vector dense
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=padding_idx)
        
        # 2. Lớp RNN: Xử lý chuỗi vector và tạo ra hidden state tại mỗi bước
        self.rnn = nn.RNN(embedding_dim, hidden_dim, batch_first=True)
        
        # 3. Lớp Linear: Ánh xạ từ hidden state sang không gian số lượng nhãn
        self.linear = nn.Linear(hidden_dim, num_tags)
    
    def forward(self, sentence):
        embeds = self.embedding(sentence)
        rnn_out, _ = self.rnn(embeds)
        tag_scores = self.linear(rnn_out)

        return tag_scores


# Task 4 — Huấn luyện Mô hình

**5 bước kinh điển trong mỗi batch:**
1. **Xóa gradient cũ**: `optimizer.zero_grad()`
2. **Forward pass**: Tính toán output từ input
3. **Tính loss**: So sánh output với nhãn thật
4. **Backward pass**: Tính gradient bằng lan truyền ngược
5. **Cập nhật trọng số**: `optimizer.step()`

In [20]:
import torch

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Device: {device}")

# 1. KHỞI TẠO MÔ HÌNH

vocab_size = len(word_to_ix)
embedding_dim = 100
hidden_dim = 128
num_tags = len(tag_to_ix)

model = SimpleRNNForTokenClassification(
    vocab_size=vocab_size,
    embedding_dim=embedding_dim,
    hidden_dim=hidden_dim,
    num_tags=num_tags,
    padding_idx=0  # <PAD> token có index = 0
).to(device)

# Khởi tạo optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Khởi tạo loss function
criterion = nn.CrossEntropyLoss(ignore_index=-100)

Device: cpu


In [33]:
# 2. VIẾT HÀM HUẤN LUYỆN

def train_epoch(model, loader, optimizer, criterion, device):

    model.train() 
    total_loss = 0.0
    num_batches = 0
    
    for batch_idx, (words_padded, tags_padded, lengths) in enumerate(loader):
        # Chuyển dữ liệu sang device
        words_padded = words_padded.to(device)
        tags_padded = tags_padded.to(device)
        
        # 5 BƯỚC KINH ĐIỂN:
        
        # Bước 1: Xóa gradient cũ
        optimizer.zero_grad()
        
        # Bước 2: Forward pass - Tính toán output
        logits = model(words_padded)
        
        # Bước 3: Tính loss
        batch_size, seq_len, num_tags_out = logits.size()
        loss = criterion(logits.view(-1, num_tags_out), tags_padded.view(-1))
        
        # Bước 4: Backward pass - Tính gradient
        loss.backward()
        
        # Bước 5: Cập nhật trọng số
        optimizer.step()
        
        # Lưu loss
        total_loss += loss.item()
        num_batches += 1
        
        # In loss mỗi 100 batches
        if (batch_idx + 1) % 100 == 0:
            print(f"  Batch {batch_idx + 1}/{len(loader)}: loss = {loss.item():.4f}")
    
    avg_loss = total_loss / num_batches
    return avg_loss


# Task 5 — Đánh giá Mô hình

In [34]:
# 1. VIẾT HÀM ĐÁNH GIÁ

def evaluate(model, loader, device, tag_pad_idx=-100):

    model.eval()  
    total_tokens = 0
    correct_predictions = 0
    
    # Tắt việc tính toán gradient để tiết kiệm bộ nhớ
    with torch.no_grad():
        for words_padded, tags_padded, lengths in loader:
            # Chuyển dữ liệu sang device
            words_padded = words_padded.to(device)
            tags_padded = tags_padded.to(device)
            
            # Forward pass
            logits = model(words_padded)  # (batch_size, seq_len, num_tags)
            
            # Lấy dự đoán bằng argmax trên chiều cuối cùng
            predictions = torch.argmax(logits, dim=-1)  # (batch_size, seq_len)
            
            # Tạo mask để bỏ qua padding tokens
            mask = tags_padded != tag_pad_idx
            
            # Đếm số token dự đoán đúng (không tính padding)
            correct_predictions += ((predictions == tags_padded) & mask).sum().item()
            total_tokens += mask.sum().item()
    
    # Tính accuracy
    accuracy = correct_predictions / total_tokens if total_tokens > 0 else 0.0
    return accuracy


In [ ]:
# 2. VÒNG LẶP HUẤN LUYỆN VÀ BÁO CÁO KẾT QUẢ

print("="*80)
print("BẮT ĐẦU HUẤN LUYỆN")
print("="*80)

num_epochs = 10
best_dev_acc = 0.0
best_epoch = 0

# Lưu lịch sử training
history = {
    'train_loss': [],
    'train_acc': [],
    'dev_acc': []
}

for epoch in range(1, num_epochs + 1):
    print(f"\n📅 Epoch {epoch}/{num_epochs}")
    print("-" * 80)
    
    # Huấn luyện
    train_loss = train_epoch(model, train_loader, optimizer, criterion, device)
    
    # Đánh giá trên tập train
    train_acc = evaluate(model, train_loader, device, tag_pad_idx=-100)
    
    # Đánh giá trên tập dev
    dev_acc = evaluate(model, dev_loader, device, tag_pad_idx=-100)
    
    # Lưu lịch sử
    history['train_loss'].append(train_loss)
    history['train_acc'].append(train_acc)
    history['dev_acc'].append(dev_acc)
    
    # In kết quả
    print(f"\n📊 Results:")
    print(f"  Train Loss:     {train_loss:.4f}")
    print(f"  Train Accuracy: {train_acc:.4f} ({train_acc*100:.2f}%)")
    print(f"  Dev Accuracy:   {dev_acc:.4f} ({dev_acc*100:.2f}%)")
    
    # Lưu mô hình tốt nhất
    if dev_acc > best_dev_acc:
        best_dev_acc = dev_acc
        best_epoch = epoch
        print(f"  🎉 New best model!")
        torch.save(model.state_dict(), 'best_pos_tagger.pt')

print("\n" + "="*80)
print("✅ HOÀN THÀNH HUẤN LUYỆN")
print("="*80)
print(f"\n🏆 KẾT QUẢ CUỐI CÙNG:")
print(f"  Best Dev Accuracy: {best_dev_acc:.4f} ({best_dev_acc*100:.2f}%)")
print(f"  Achieved at Epoch: {best_epoch}")
print("="*80)



BẮT ĐẦU HUẤN LUYỆN

Epoch 1/10
--------------------------------------------------------------------------------
  Batch 100/392: loss = 0.0492
  Batch 100/392: loss = 0.0492
  Batch 200/392: loss = 0.0676
  Batch 200/392: loss = 0.0676
  Batch 300/392: loss = 0.0672
  Batch 300/392: loss = 0.0672

 Đánh giá trên tập train...

 Đánh giá trên tập train...
 Đánh giá trên tập dev...

Epoch 1 Results:
  • Train Loss:     0.0650
  • Train Accuracy: 0.9851 (98.51%)
  • Dev Accuracy:   0.8754 (87.54%)
New best model. Saving...

Epoch 2/10
--------------------------------------------------------------------------------
 Đánh giá trên tập dev...

Epoch 1 Results:
  • Train Loss:     0.0650
  • Train Accuracy: 0.9851 (98.51%)
  • Dev Accuracy:   0.8754 (87.54%)
New best model. Saving...

Epoch 2/10
--------------------------------------------------------------------------------
  Batch 100/392: loss = 0.0553
  Batch 100/392: loss = 0.0553
  Batch 200/392: loss = 0.0536
  Batch 200/392: loss = 0.

KeyboardInterrupt: 

In [ ]:
# 3. VẼ ĐỒ THỊ TRAINING HISTORY

import matplotlib.pyplot as plt

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))

# Plot Loss
ax1.plot(range(1, num_epochs + 1), history['train_loss'], marker='o', label='Train Loss')
ax1.set_xlabel('Epoch')
ax1.set_ylabel('Loss')
ax1.set_title('Training Loss')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Plot Accuracy
ax2.plot(range(1, num_epochs + 1), history['train_acc'], marker='o', label='Train Accuracy')
ax2.plot(range(1, num_epochs + 1), history['dev_acc'], marker='s', label='Dev Accuracy')
ax2.set_xlabel('Epoch')
ax2.set_ylabel('Accuracy')
ax2.set_title('Training & Dev Accuracy')
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("📈 Đồ thị training history đã được vẽ!")


In [ ]:
# 4. HÀM DỰ ĐOÁN CHO CÂU MỚI (NÂNG CAO)

def predict_sentence(sentence_text, model, word_to_ix, tag_to_ix, device):
    """
    Dự đoán POS tags cho một câu mới.
    
    Args:
        sentence_text: Câu cần dự đoán (string), ví dụ: "The cat is sleeping"
        model: Mô hình đã train
        word_to_ix: Dict ánh xạ word → index
        tag_to_ix: Dict ánh xạ tag → index
        device: Device (CPU/GPU)
        
    Returns:
        predictions: List of (word, predicted_tag) tuples
    """
    model.eval()
    
    # Tạo reverse mapping: index → tag
    ix_to_tag = {v: k for k, v in tag_to_ix.items()}
    
    # Xử lý câu input
    words = sentence_text.lower().split()
    
    # Chuyển words thành indices
    word_indices = []
    for word in words:
        # Nếu từ không có trong vocab, dùng <UNK>
        word_idx = word_to_ix.get(word, word_to_ix.get('<UNK>', 1))
        word_indices.append(word_idx)
    
    # Chuyển thành tensor
    input_tensor = torch.LongTensor([word_indices]).to(device)  # Shape: (1, seq_len)
    
    # Dự đoán
    with torch.no_grad():
        logits = model(input_tensor)  # Shape: (1, seq_len, num_tags)
        predictions = torch.argmax(logits, dim=-1)  # Shape: (1, seq_len)
    
    # Chuyển predictions về CPU và lấy list
    predicted_tags = predictions[0].cpu().tolist()
    
    # Decode về tag names
    predicted_tag_names = [ix_to_tag.get(tag_idx, '<UNK>') for tag_idx in predicted_tags]
    
    # Tạo kết quả
    result = list(zip(words, predicted_tag_names))
    
    return result

print("✅ Đã định nghĩa hàm predict_sentence()")


In [ ]:
# TEST HÀM DỰ ĐOÁN

print("\n" + "="*80)
print("TEST DỰ ĐOÁN TRÊN CÂU MỚI")
print("="*80)

# Các câu test
test_sentences = [
    "The cat is sleeping on the couch",
    "I love programming in Python",
    "She quickly ran to the store",
    "The beautiful flowers are blooming in the garden"
]

for sent in test_sentences:
    print(f"\n📝 Câu: {sent}")
    predictions = predict_sentence(sent, model, word_to_ix, tag_to_ix, device)
    print(f"🏷️  Dự đoán:")
    for word, tag in predictions:
        print(f"  {word:15s} → {tag}")

print("\n" + "="*80)
